In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup

import nltk
import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

In [2]:
file1 = 'data/QueryResults3_new.csv'
df1 = pd.read_csv(file1,index_col=['Id'])

import mes_fonctions_final2 as stt

In [3]:
StopWord = stt.load_stop_word()

## Chargement des Tags

In [4]:
Y_tags, Y_tags_voc = stt.genere_target_dummy_and_vocabulary(data=df1, min_df=300)
Tags_freq = stt.genere_df_target_tags_freq(data=df1, min_df=300)

/!\ Ajout des features en cours


## Filtre de X_train 

In [5]:
# On sépare les données d'entrainement de celles de tests
debut = datetime.datetime.now()
X = df1['Body']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y_tags, test_size=0.2, random_state=42 )
fin = datetime.datetime.now()
print("Fin Train_Split : {}".format(fin-debut))

Fin Train_Split : 0:00:00.068843


## Séparation Title / Body

In [6]:
# VERSION TITLE
X_train_Title = df1.loc[X_train.index, 'Title']
X_test_Title = df1.loc[X_test.index, 'Title']

train_title = pd.DataFrame(X_train_Title)
test_title = pd.DataFrame(X_test_Title)
train_title.columns = ['post']
test_title.columns = ['post']

# VERSION BODY
X_train_body = df1.loc[X_train.index, 'Body']
X_test_body = df1.loc[X_test.index, 'Body']

train_body = pd.DataFrame(X_train_body)
test_body = pd.DataFrame(X_test_body)
train_body.columns = ['post']
test_body.columns = ['post']

## ELMO 

#### Librairies

In [8]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn import preprocessing
import keras
import numpy as np

Using TensorFlow backend.


#### On Charge le modèle pré-entrainé d'ELMO

In [9]:
url = "https://tfhub.dev/google/elmo/2"
embed = hub.Module(url)

INFO:tensorflow:Using /var/folders/95/fydcf18x5rz7wljn4wqt_tq00000gn/T/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 60.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 120.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 180.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 230.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 290.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 350.35MB
INFO:tensorflow:Downloaded https://tfhub.dev/google/elmo/2, Total size: 357.40MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


##### On encode les features cibles : 
> uniquement sur python

In [10]:
## Y 
y1 = np.where(Y_train['python'].to_dense()==0, 'non', 'python')
y2 = np.where(Y_test['python'].to_dense()==0, 'non', 'python')

le = preprocessing.LabelEncoder()
le.fit(y1)

def encode(le, labels):
    enc = le.transform(labels)
    return keras.utils.to_categorical(enc)

def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)

y_train = np.asarray(encode(le, y1))
y_test = np.asarray(encode(le, y2))

In [11]:
## On transforme les X en np.array

x_train_title = np.asarray(X_train_Title)
x_train_body = np.asarray(X_train_body)

x_test_title = np.asarray(X_test_Title)
x_test_body = np.asarray(X_test_body)

## Paramétrage des MODEL

##### Partie Title

In [12]:
from keras.layers import Input, Lambda, Dense
from keras.models import Model
import keras.backend as K

def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

input_text_title = Input(shape=(1,), dtype=tf.string)
embedding_title = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text_title)
dense_title = Dense(256, activation='relu')(embedding_title)
pred_title = Dense(2, activation='softmax')(dense_title)
model_title = Model(inputs=[input_text_title], outputs=pred_title)
model_title.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


##### Partie Body

In [13]:
input_text_body = Input(shape=(1,), dtype=tf.string)
embedding_body = Lambda(ELMoEmbedding, output_shape=(1024, ))(input_text_body)
dense_body = Dense(256, activation='relu')(embedding_body)
pred_body = Dense(2, activation='softmax')(dense_body)
model_body = Model(inputs=[input_text_body], outputs=pred_body)
model_body.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Entrainement des 2 modèles

### Entrainement du RNN pour le Body

In [14]:
%%time 
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    ####history_body = model_body.fit(x_train_body, y_train, epochs=1, batch_size=32)
    history_body = model_body.fit(x_train_body, y_train, epochs=1, batch_size=8)
    model_body.save_weights('./elmo-model-body.h5')

Epoch 1/1
40000/40000 [==============================] - 53262s 1s/step - loss: 0.2883 - acc: 0.8970
CPU times: user 4d 41min 41s, sys: 2h 27min 9s, total: 4d 3h 8min 50s
Wall time: 14h 47min 42s


> L'Entrainement aura duré 18h 

### Entrainement du RNN pour le Titre

In [ ]:
%%time 
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    history_title = model_title.fit(x_train_title, y_train, epochs=1, batch_size=32)
    model_title.save_weights('./elmo-model-title.h5')

> L'Entrainement est ici beaucoup plus rapide : 36 min car il y a beaucoup moins de mots

# Prédiction & Test

##### Title

In [64]:
%%time
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model_title.load_weights('./elmo-model-title.h5')  
    predicts_title = model_title.predict(x_test_title, batch_size=32)

CPU times: user 39min 43s, sys: 42.3 s, total: 40min 26s
Wall time: 6min 14s


In [65]:
y_preds_title2 = decode(le, predicts_title)

df_y_test_title = pd.DataFrame(np.where(y_test=='non', 0, 1), columns=['python'])
df_y_preds_title = pd.DataFrame(np.where(y_preds_title2=='non', 0, 1), columns=['python'])

/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


##### Body

In [17]:
%%time 
with tf.Session() as session:
    K.set_session(session)
    session.run(tf.global_variables_initializer())
    session.run(tf.tables_initializer())
    model_body.load_weights('./elmo-model-body.h5')  
    predicts_body = model_body.predict(x_test_body, batch_size=8)

CPU times: user 22h 48min 18s, sys: 35min 50s, total: 23h 24min 8s
Wall time: 3h 29min 57s


> La prédiction du Body est aussi longue avec le plongement : 3h30 

In [19]:
predicts = predicts_body.copy()

#### Mesure

In [20]:
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

from sklearn import metrics

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))


[[8729  154]
 [ 763  354]]
             precision    recall  f1-score   support

        non       0.92      0.98      0.95      8883
     python       0.70      0.32      0.44      1117

avg / total       0.89      0.91      0.89     10000



/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/seb/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


##### On fait une sauvegarde intermédiaire des données pour pouvoir les analyser après coup

In [43]:
#df_y_test_body = pd.DataFrame(y_test)
#df_y_preds_body = pd.DataFrame(y_preds)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

df_y_test_body = pd.DataFrame(np.where(y_test=='non', 0, 1), columns=['python'])
df_y_preds_body = pd.DataFrame(np.where(y_preds=='non', 0, 1), columns=['python'])

In [69]:
df_y_body_OR_title = pd.DataFrame(
    np.where(np.logical_or(df_y_preds_title, df_y_preds_body), 1, 0),
    columns=df_y_preds_title.columns, index=df_y_preds_title.index)

In [86]:
df_y_test_body.to_csv('ELMO_y_test_body_python.csv', index=True)
df_y_preds_title.to_csv('ELMO_y_preds_title_python.csv', index=True)
df_y_preds_body.to_csv('ELMO_y_preds_body_python.csv', index=True)
df_y_body_OR_title.to_csv('ELMO_y_preds_title_OR_body_python.csv', index=True)

## Mesure

In [81]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, roc_auc_score, f1_score
from sklearn.metrics import confusion_matrix

l_tags = ['python']
m_ELMO1_body = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])
m_ELMO1_title = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])
m_ELMO1_title_OR_body = pd.DataFrame(index=l_tags, 
                        columns=['f1_score', 'recall', 'accuracy_score'])

for i_y_pred, col in enumerate(l_tags):
    # Title
    m_ELMO1_title.loc[col, 'f1_score'] = f1_score(
        y_true=df_y_test_title[col].ravel(), 
        y_pred=df_y_preds_title[col].to_dense().ravel())
    
    m_ELMO1_title.loc[col, 'recall'] = recall_score(
        y_true=df_y_test_title[col].ravel(), 
        y_pred=df_y_preds_title[col].to_dense().ravel())
    
    m_ELMO1_title.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=df_y_test_title[col].ravel(), 
        y_pred=df_y_preds_title[col].to_dense().ravel())
    
    # Body
    m_ELMO1_body.loc[col, 'f1_score'] = f1_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_preds_body[col].to_dense().ravel())
    
    m_ELMO1_body.loc[col, 'recall'] = recall_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_preds_body[col].to_dense().ravel())
    
    m_ELMO1_body.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_preds_body[col].to_dense().ravel())
    
    # Title OR Body
    m_ELMO1_title_OR_body.loc[col, 'f1_score'] = f1_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_body_OR_title[col].to_dense().ravel())
    
    m_ELMO1_title_OR_body.loc[col, 'recall'] = recall_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_body_OR_title[col].to_dense().ravel())
    
    m_ELMO1_title_OR_body.loc[col, 'accuracy_score'] = accuracy_score(
        y_true=df_y_test_body[col].ravel(), 
        y_pred=df_y_body_OR_title[col].to_dense().ravel())
    
Total_python = pd.concat([m_ELMO1_title, m_ELMO1_body, m_ELMO1_title_OR_body], )
Total_python.index=['python_title', 'python_body', 'python_OR']
Total_python

,f1_score,recall,accuracy_score
python_title,0.522262,0.388541,0.9206
python_body,0.435692,0.31692,0.9083
python_OR,0.634171,0.564906,0.9272


> Les résultats sont assez décevants par rapport aux autres méthodes mais on reste sur des valeurs qui sont tout de même bonnes